In [1]:
from featureExtraction import featureExtraction
from modelGenerator import modelGenerator
import pandas as pd
import matplotlib.pyplot as plt
from utils import reset_seeds


reset_seeds(seed_value=39)
dataID = "raw_data_10000_samples_fm_20000_tests_Prueba_21_Prueba_24_Prueba_27"
modelIDs = ["linearSGD","svr","mlp"]
params = {}

data = featureExtraction(dataID,statorFreqs=[37],testsID=[21])  # raw_data_10000_samples_fm_20000_tests_Prueba_21_Prueba_24_Prueba_27
dataTest = featureExtraction(dataID,statorFreqs=[37],testsID=[24],scaler_params=data.get_scaler_params())  # raw_data_10000_samples_fm_20000_tests_Prueba_21_Prueba_24_Prueba_27
MODELS = []

for modelID in modelIDs:
    model = modelGenerator(modelID=modelID, data=data,params=params)
    model.train()
    MODELS.append(model)

2023-07-08 13:42:49.220521: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/diego/.conda/envs/hiwind/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.0.3) or chardet (None)/charset_normalizer (2.1.1) doesn't match a supported version!
  warnings.warn(


training <class 'models.linearSGD.linearSGD'>
Test MSE: 4.901320469027718    Test MAE: 1.8583893799847278  
Test MSE: 1.4039918118314543    Test MAE: 0.9570821180950091  


/home/diego/.conda/envs/hiwind/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=300).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
2023-07-08 13:42:57.258753: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-08 13:42:57.534602: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22309 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 6)]               0         
                                                                 
 dense_0 (Dense)             (None, 130)               910       
                                                                 
 dense_1 (Dense)             (None, 140)               18340     
                                                                 
 dense (Dense)               (None, 1)                 141       
                                                                 
Total params: 19,391
Trainable params: 19,391
Non-trainable params: 0
_________________________________________________________________
Epoch 1/300
127/175 [====================>.........] - ETA: 0s - loss: 14660.4424 - mean_squared_error: 14660.4424 - mean_absolute_error: 117.4809

2023-07-08 13:42:58.181869: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


175/175 [==============================] - 1s 1ms/step - loss: 13622.6875 - mean_squared_error: 13622.6875 - mean_absolute_error: 112.7915 - val_loss: 9543.0479 - val_mean_squared_error: 9543.0479 - val_mean_absolute_error: 93.6938
Epoch 2/300
175/175 [==============================] - 0s 978us/step - loss: 7194.4712 - mean_squared_error: 7194.4712 - mean_absolute_error: 79.9705 - val_loss: 5026.0435 - val_mean_squared_error: 5026.0435 - val_mean_absolute_error: 66.0147
Epoch 3/300
175/175 [==============================] - 0s 972us/step - loss: 3789.1746 - mean_squared_error: 3789.1746 - mean_absolute_error: 56.7188 - val_loss: 2625.0796 - val_mean_squared_error: 2625.0796 - val_mean_absolute_error: 46.5579
Epoch 4/300
175/175 [==============================] - 0s 967us/step - loss: 2039.2649 - mean_squared_error: 2039.2649 - mean_absolute_error: 41.0430 - val_loss: 1462.6588 - val_mean_squared_error: 1462.6588 - val_mean_absolute_error: 34.6811
Epoch 5/300
175/175 [==================

In [2]:
X_test = dataTest.X
y_test = dataTest.y
Y_test_est = []
for model in MODELS:
    Y_test_est.append(model.predict(X_test))

9/9 [==============================] - 0s 546us/step


In [3]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import numpy as np
output_notebook()
colors = ["royalblue","sandybrown","seagreen"]
MODEL_LABELS = {"mlp": "MLP","svr": "SVR", "linearSGD":"MLR"}

# filtering some temperature range
min_temp = 130
idx_fig = y_test>min_temp
y_test_fig = y_test[idx_fig]
Y_test_est_fig = [Y_test_est[y][idx_fig] for y in range(len(modelIDs))]

p = figure(width=600,height=450)
p.line(np.arange(len(y_test_fig)),y_test_fig, line_width=4,line_alpha=0.8, line_color='gray', legend_label="Ground truth" )
for ii,model in enumerate(MODELS):
    p.line(np.arange(len(y_test_fig)),Y_test_est_fig[ii], line_width=2,  line_color=colors[ii],line_alpha=0.6,legend_label=MODEL_LABELS[modelIDs[ii]])
p.legend.location = 'bottom_right'
p.yaxis.axis_label = r"T stator (ºC)"
p.yaxis.axis_label_text_font_size = "13pt"
p.yaxis.major_label_text_font_size = "10pt"
p.xaxis.axis_label = r"sample"
p.xaxis.axis_label_text_font_size = "13pt"
p.xaxis.major_label_text_font_size = "10pt"
p.legend.label_text_font_size = "13pt"

show(p)


Loading BokehJS ...

In [4]:
from bokeh.io import export_png
export_png(p,filename="freqModelsTemp.png")

RuntimeError: Neither firefox and geckodriver nor a variant of chromium browser and chromedriver are available on system PATH. You can install the former with 'conda install -c conda-forge firefox geckodriver'.

In [5]:
$PAHT

SyntaxError: invalid syntax (572727887.py, line 1)